In [79]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math
import sounddevice as sd

In [80]:
detector = HandDetector(detectionCon=0.8, maxHands=2)

In [81]:
GESTURE = {
    "bass": [0, 0, 0, 0, 0],
    "mid": [0, 1, 0, 0, 0],
    "treble": [0, 0, 0, 0, 1],
    "all": [1, 1, 1, 1, 1],
    "toggle": [1, 1, 0, 0, 1]  # gesture to turn on/off adjustment mode
}

In [82]:
EQ_RANGE = [0, 100]
VOLUME_RANGE = (-50, 50)
INITIAL_VOLUME_ANGLE = 60

In [ ]:
audio_file = 'playlist/Butterfly.mp3'

In [84]:
import os
if os.path.exists(audio_file):
    print("File exists!")
else:
    print("File not found.")

File not found.


In [85]:
from pydub.utils import which
print(which("ffmpeg"))


C:\Users\Meng\AppData\Local\Microsoft\WinGet\Packages\Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe\ffmpeg-7.1.1-full_build\bin\ffmpeg.exe


In [86]:
from EqController import EQController
from AudioPlayer import AudioPlayer

eq_controller = EQController()
player = AudioPlayer(audio_file, eq_controller)

adjust_mode = False
prev_toggle_state = False  # to prevent flipping too fast
toggle_cooldown = 0


d:\School\OneDrive - Fulbright University Vietnam\ComVi\eq-hand\.venv\Lib\site-packages\pydub\utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
d:\School\OneDrive - Fulbright University Vietnam\ComVi\eq-hand\.venv\Lib\site-packages\pydub\utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
d:\School\OneDrive - Fulbright University Vietnam\ComVi\eq-hand\.venv\Lib\site-packages\pydub\utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
d:\School\OneDrive - Fulbright University Vietnam\ComVi\eq-hand\.venv\Lib\site-packages\pydub\utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


FileNotFoundError: [Errno 2] No such file or directory: 'Butterfly.mp3'

In [ ]:
import threading
threading.Thread(target=player.play).start()

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

while True:
    success, img = cap.read()
    if not success:
        break

    hands, img = detector.findHands(img)

    if len(hands) == 2:
        hand1, hand2 = hands
        if hand1["center"][0] < hand2["center"][0]:
            left_hand = hand2
            right_hand = hand1
        else:
            left_hand = hand1
            right_hand = hand2

        fingers_left = detector.fingersUp(left_hand)
        # == Adjustment Mode ==
        if fingers_left == GESTURE["toggle"]:
            if not prev_toggle_state and toggle_cooldown == 0:
                adjust_mode = not adjust_mode
                toggle_cooldown = 20 # frames to prevent spamming toggle
            prev_toggle_state = True
        else:
            prev_toggle_state = False

        if toggle_cooldown > 0:
            toggle_cooldown -= 1

        # == EQ and Volume Control ==
        freq_band = "none" # default
        gain = 1.0 # default
        volume = 1.0 # default
        if adjust_mode:
            for control, gesture in GESTURE.items():
                if fingers_left == gesture:
                    freq_band = control

                    if control == "all": # adjust the volume
                        wrist = np.array(right_hand['lmList'][0])
                        middle_tip = np.array(right_hand['lmList'][12])

                        dx = middle_tip[0] - wrist[0]
                        dy = middle_tip[1] - wrist[1]
                        angle = -math.degrees(math.atan2(dy, dx)) - INITIAL_VOLUME_ANGLE
                        angle_clamped = max(
                            VOLUME_RANGE[0], min(VOLUME_RANGE[1], angle))

                        volume = (angle_clamped - VOLUME_RANGE[0]) / (VOLUME_RANGE[1] - VOLUME_RANGE[0])
                        player.set_volume(volume)
                        player.get_volume()


                    else: # adjust the eq
                        distance, info, img = detector.findDistance(
                            right_hand["lmList"][8][:-1],
                            right_hand["lmList"][4][:-1],
                            img
                        )
                        gain = max(EQ_RANGE[0], min(EQ_RANGE[1], int(distance * 0.5)))/100
                        # set band name
                        eq_controller.set_band(control)
                        eq_controller.set_gain(gain)
                        # assign band name and gain
                        freq_band = eq_controller.get_selected_band()
                        gain = eq_controller.get_gains()
                    break

        cv2.putText(img, f"Freq Band: {freq_band}", (10, 70),
                    cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3,)
        cv2.putText(img, f"Gain: {gain}", (10, 110),
                    cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3,)
        cv2.putText(img, f"Volume: {int(volume*100)} %", (10, 150),
                    cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3,)
        cv2.putText(img, f"Adjust Mode: {'ON' if adjust_mode else 'OFF'}", (10, 190),
            cv2.FONT_HERSHEY_PLAIN, 3, (0, 255, 0) if adjust_mode else (0, 0, 255), 3)

    cv2.imshow("Hand Gesture Equalizer Control", img)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        player.stop()
        break

cap.release()
cv2.destroyAllWindows()